In [43]:
import os
import polars as pl
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
#import pyarrow as pa
#import datetime

# Define paths
wea_files = ['LC_2022Q1.csv', 'LC_2022Q2.csv', 'LC_2022Q3.csv', 'LC_2022Q4.csv']
mp_list = ['MP 01', 'MP 02', 'MP 03', 'MP 04', 'MP 05', 'MP 06', 'MP 07', 'MP 08']
id_list = ['439', '440', '441', '910', '442', '443', '444', '324'] # Same order as 1-9 in project.pdf
#id_list = ['439', '440']

# Load with polars and remove noise stations 7 8 9
noise_loc = pl.read_csv('../noise_meta.csv')\
            .slice(0,6)
wea_loc = pl.read_csv('../01_Metadata_v2.csv')
wea_prep = pd.read_csv('out/wea_prep.csv')

In [49]:
# Preprocess noise level with pandas
# Using output_path fixes [Errno 13]
def noise_preprocess_csv(folder_path, output_path, names):
    noise = pd.DataFrame()
    for name in names:
        noise_row = pd.DataFrame()
        for file_name in os.listdir(folder_path):
            if file_name.endswith('.csv') and name in file_name:
                file_path = os.path.join(folder_path, file_name)
                add = pd.read_csv(file_path, delimiter=';')

                add['date'] = add['result_timestamp'].str[0:10]
                add = add.groupby(['description', 'date'])['lamax'].mean().reset_index()

                add = add.pivot(index='description', columns='date', values='lamax')

                noise_row = pd.concat([noise_row, add], axis=1, sort=True)
        noise = pd.concat([noise, noise_row], axis=0, sort=False)
        noise['description'] = mp_list
    noise.to_csv(os.path.join(output_path, 'noise-by-day2.csv'), index=False)

    return 0

In [ ]:
noise_preprocess_csv('../noise level/', 'out/', id_list)